In [1]:
from dotenv import load_dotenv
import os
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.astra_db import AstraDBVectorStore

from llama_index.llms.groq import Groq

d:\Github\rag\ingestion-pipeline\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
load_dotenv()

astra_api_endpoint = os.getenv('ASTRA_DB_ENDPOINT')
astra_db_token = os.getenv('ASTRA_DB_TOKEN')
groq_api_token = os.getenv('GROQ_API_TOKEN')

# Validate that all required environment variables are present
if not all([astra_api_endpoint, astra_db_token, groq_api_token]):
    raise ValueError("Missing required environment variables. Please check your .env file")

In [3]:
# load documents
documents = SimpleDirectoryReader("./data/medical/").load_data(num_workers=10)
print(f"Total documents: {len(documents)}")
print(f"First document, id: {documents[0].doc_id}")
print(f"First document, hash: {documents[0].hash}")
print(
    "First document, text"
    f" ({len(documents[0].text)} characters):\n{'='*20}\n{documents[0].text[:360]} ..."
)

Total documents: 1570
First document, id: 71a7d1ce-2d6e-4eac-af07-0847a2182e5e
First document, hash: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
First document, text (0 characters):
 ...


In [5]:
astra_db_store = AstraDBVectorStore(
    token=astra_db_token,
    api_endpoint=astra_api_endpoint,
    collection_name="medical_collections",
    embedding_dimension=768,
)

In [6]:
embed_model = OllamaEmbedding(model_name="nomic-embed-text")

storage_context = StorageContext.from_defaults(vector_store=astra_db_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [7]:
llm = Groq(model="deepseek-r1-distill-llama-70b", api_key=groq_api_token)

query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("A BRIEF REVIEW OF THE VERTEBRAL COLUMN")

print(response.response)

<think>
Okay, I need to answer the query "A BRIEF REVIEW OF THE VERTEBRAL COLUMN" using the provided context information without referencing it directly. Let me go through the context step by step.

First, looking at page 135 from Snell's Neuroanatomy, it lists the different sections of the vertebral column: cervical (7 vertebrae), thoracic (12), lumbar (5), sacral (5 fused), and coccygeal (4 fused). It also mentions the curves—cervical, thoracic, lumbar, sacral. So I should include the number of vertebrae and the curves.

Next, the context describes various parts of the vertebrae, like the vertebral foramen, lamina, pedicle, transverse process, body, etc. It also talks about the ligaments: anterior and posterior longitudinal ligaments, their attachments, and strengths. I should summarize these structural components and the ligaments.

The second context from page 114 discusses the spinal cord's cell columns, but since the query is about the vertebral column, I can focus on the first c

In [8]:
llm = Groq(model="deepseek-r1-distill-llama-70b", api_key=groq_api_token)

query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("explain in great detail the cranial cavity")

print(response.response)

<think>
Okay, I need to explain the cranial cavity in great detail using the provided context. Let me start by looking through the context information.

First, I see that the cranial cavity is mentioned in page 83 as the location of the brain. It's part of the central nervous system along with the spinal cord, which is in the vertebral canal. The CNS is protected by the skull, vertebral canal, meninges, and cerebrospinal fluid.

Looking at page 246, the cerebrum is described as filling most of the cranial cavity. It's the largest part of the brain, with two cerebral hemispheres connected by the corpus callosum. The longitudinal cerebral fissure separates the hemispheres and contains structures like the falx cerebri, arachnoid fold, pia mater, and blood vessels.

I should structure the answer to cover the structure, contents, protection mechanisms, and functions. I'll mention the bones forming the cranial cavity, the meninges layers, the CSF, and the brain's composition of grey and whit